# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [2]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [3]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [4]:
deals = ScrapedDeal.fetch(show_progress=True)

 20%|██        | 1/5 [00:26<01:47, 26.95s/it]/Users/shengy/Documents/GitHub/llm_engineering/week8/agents/deals.py:27: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  description = BeautifulSoup(description, 'html.parser').get_text()
100%|██████████| 5/5 [02:58<00:00, 35.69s/it]


In [5]:
len(deals)

50

In [6]:
deals[44].describe()

"Title: Lowe's The Back Aisle Clearance Flooring Deals from $1/ sq. ft. + free shipping w/ $35\nDetails: Shop flooring deals in a variety of sizes and colors. Choose in-store pickup to dodge the $5.99 shipping fee, or get free shipping with orders of $35 or more by signing up to MyLowe's Rewards. (It's free to join.) Buy Now at Lowe's\nFeatures: \nURL: https://www.dealnews.com/Lowes-The-Back-Aisle-Clearance-Flooring-Deals-from-1-sq-ft-free-shipping-w-35/21719065.html?iref=rss-c196"

In [7]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [8]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [9]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Samsung Certified Re-Newed Smartphones: At least $300 off w/ trade in good condition + free shipping
Details: Save on Galaxy S21, S22, or S23 when you trade your current device in good condition. You can also get $150 off if you have a device with a cracked screen. Shop Now at Samsung
Features: 
URL: https://www.dealnews.com/Samsung-Certified-Re-Newed-Smartphones-At

In [10]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [11]:
result = get_recommendations()

In [12]:
len(result.deals)

5

In [13]:
result.deals[1]

Deal(product_description='Experience stunning visuals with the LG evo C4 Series OLED65C4PUA, a 65-inch 4K UHD Smart TV. It features Dolby Vision, HDR 10 for enhanced picture quality, and a 120Hz refresh rate for smooth motion. This smart TV integrates seamlessly with Amazon Alexa, Google Assistant, and Apple HomeKit, offering easy control and connectivity. Multiple HDMI and USB ports provide flexibility for connecting various devices.', price=1198.0, url='https://www.dealnews.com/products/LG/LG-evo-C4-Series-OLED65-C4-PUA-65-4-K-OLED-UHD-Smart-TV/472547.html?iref=rss-f1912')

In [14]:
from agents.scanner_agent import ScannerAgent

In [15]:
agent = ScannerAgent()
result = agent.scan()

In [16]:
result

DealSelection(deals=[Deal(product_description='The Certified Refurb EcoFlow Delta Pro is a powerful portable power station with a 3,600Wh capacity, designed to provide reliable energy solutions for both home and outdoor activities. It features multiple output ports, including USB, AC, and DC, allowing you to charge various devices simultaneously. This unit is certified refurbished, ensuring it meets quality standards while being an eco-friendly option. It comes with a 2-year Allstate warranty for added peace of mind.', price=1349.0, url='https://www.dealnews.com/products/Eco-Flow/Eco-Flow-Delta-Pro-3-600-Wh-Power-Station/472609.html?iref=rss-c142'), Deal(product_description='The Samsung Galaxy Watch7 is an advanced smartwatch that comes in two sizes: 40mm and 44mm, catering to different wrist sizes and personal preferences. It features an AMOLED display, fitness tracking capabilities, and customizable watch faces to suit your style. The device integrates seamlessly with your smartphone

In [17]:
type(result)

agents.deals.DealSelection

In [18]:
result.deals[0]

Deal(product_description='The Certified Refurb EcoFlow Delta Pro is a powerful portable power station with a 3,600Wh capacity, designed to provide reliable energy solutions for both home and outdoor activities. It features multiple output ports, including USB, AC, and DC, allowing you to charge various devices simultaneously. This unit is certified refurbished, ensuring it meets quality standards while being an eco-friendly option. It comes with a 2-year Allstate warranty for added peace of mind.', price=1349.0, url='https://www.dealnews.com/products/Eco-Flow/Eco-Flow-Delta-Pro-3-600-Wh-Power-Station/472609.html?iref=rss-c142')